### Front End Visualization

In [ ]:
from bokeh.plotting import figure, show

from bokeh.layouts import column
from bokeh.models import ColumnDataSource, CustomJS

import json

# Define a callback function to update the data source
def update_data():
    api = API0x(chain = '', api_key='2962063a-97d3-4f86-8cea-461485d34941')  
    data_json = api.apply("TOKEN_A", "TOKEN_B", 1.0)
    data = json.loads(data_json)
    source.data = data


# Create a ColumnDataSource
source = ColumnDataSource(data=dict(timestamp=[], price=[]))

# Create a line plot
p = figure(
    x_axis_label="Timestamp",
    y_axis_label="Price",
    title="Live Data Feed from API0x",
)
line = p.line("timestamp", "price", source=source, line_width=2)

# Configure callback and timer
callback = CustomJS(args=dict(source=source), code=update_data, trigger="timeout")
p.js_on_event("timeout", callback)

# Set initial timeout and run the callback
show(column(p))
cb_id = p.js_on_event("timeout", callback)
p.js_on_event("timeout", callback)  # Trigger initial update

# Set a timer to call the callback every 5 seconds
p.js_on_event("timeout", CustomJS(args=dict(callback_id=cb_id), code="""
    clearTimeout(%s);
    setTimeout(%s, 5000);
""", callback_id=cb_id))

